We will plot the three longest whiskers on each side, one side after each other

In [ ]:
import os
import cv2
from pathlib import Path
import tables
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Navigate to the directory where the video and tracking data is stored
# data_dir="E:/test/whisker_tracking"
data_dir=Path('/home/wanglab/data/whisker_asym/sc012/test')
    # '/home/wanglab/data/whisker_asym/sc014/sc014_0325/sc014_0325_002/raw_data/behav_vid'
# video_name="whiskers.mp4"
video_name="sc012_0119_001_20230119_10sWhisking.mp4"
# "sc014_0325_002.mp4"
video_file=os.path.join(data_dir,video_name)
os.chdir(data_dir)

In [ ]:
# open WT/output_right.hdf5
# h5file_right = tables.open_file(data_dir / 'WT' / 'output_right.hdf5', mode='r')
# h5file_right

# Open the combined hdf5 file
# h5file = tables.open_file(data_dir / 'sc012_0119_001.hdf5', mode='r')
# h5file

In [ ]:
# summary = h5file.get_node('/summary')
# summary[:10]

In [ ]:
import zarr
import json
# Open the combined zarr file
zarrdir = data_dir / 'sc012_0119_001.zarr'
# bu_zarrdir = data_dir / 'sc012_0119_001.zarr.bu'
# Open the Zarr file
zarr_file = zarr.open(zarrdir, mode='r')
# bu_zarr_file = zarr.open(bu_zarrdir, mode='r')

In [ ]:
# bu_zarr_file['pixels_x']
zarr_file['pixels_x']

In [ ]:
# Retrieve and print the summary table content
summary = zarr_file['summary'][:]
print("Summary Data first 5 rows:")
print(summary[:5])

# # Retrieve and print the pixels_x content
# pixels_x = [json.loads(x) for x in zarr_file['pixels_x'][:]]
# print("\nPixels X Data first 5 rows:")
# print(pixels_x[:5])

# # Retrieve and print the pixels_y content
# pixels_y = [json.loads(y) for y in zarr_file['pixels_y'][:]]
# print("\nPixels Y Data first 5 rows:")
# print(pixels_y[:5])

# Retrieve and print the pixels_x content
pixels_x = []
indices_x = zarr_file['pixels_x_indices'][:]
for start, end in indices_x[:5]:  # Get the first 5 rows
    pixels_x.append(zarr_file['pixels_x'][start:end])

print("\nPixels X Data first 5 rows:")
print(pixels_x)

# Retrieve and print the pixels_y content
pixels_y = []
indices_y = zarr_file['pixels_y_indices'][:]
for start, end in indices_y[:5]:  # Get the first 5 rows
    pixels_y.append(zarr_file['pixels_y'][start:end])

print("\nPixels Y Data first 5 rows:")
print(pixels_y)

In [ ]:
for i in range(5):
    print(np.shape(pixels_x[i]))
    print(np.shape(pixels_y[i]))

In [ ]:
# Plot the first 5 whiskers
for i in range(5):
    plt.plot(pixels_x[i], pixels_y[i])
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#  Save summary to a csv file
import pandas as pd

# summary_df = summary.read()
# summary_df = pd.DataFrame(summary_df)
# summary_df.to_csv(data_dir / 'summary_hdf5.csv', index=False)

# Retrieve the summary data
summary = zarr_file['summary'][:]

# Convert to pandas DataFrame
summary_df = pd.DataFrame(summary, columns=zarr_file['summary'].dtype.names)

# Decode byte strings in the 'face_side' field
summary_df['face_side'] = summary_df['face_side'].str.decode('utf-8')

#  Save the DataFrame to a CSV file
csv_file = data_dir / 'summary_zarr_fast.csv'
summary_df.to_csv(csv_file, index=False)

In [ ]:
summary_df

In [ ]:
# Open the HDF5 file and get the pixel data
face_side = 'left'
h5file = tables.open_file(f'{os.path.basename(video_name).split(".")[0]}_{face_side}.hdf5', mode='r')
pixels_x = h5file.get_node('/pixels_x')
pixels_y = h5file.get_node('/pixels_y')

# Get the summary data
summary = h5file.get_node('/summary')

# Get unique whisker IDs
whisker_ids = np.unique(summary[:]['wid'])

# Create set of colors for up to 20 whiskers, starting with red, green, blue
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255),
            (0,255,255), (128,0,0), (0,128,0), (0,0,128), (128,128,0),
            (128,0,128), (0,128,128), (64,0,0), (0,64,0), (0,0,64),
            (64,64,0), (64,0,64), (0,64,64), (192,0,0), (0,192,0)]

In [ ]:
# If using split data (left or right whiskers), open the corresponding whiskerpad_{side}.json file
with open(f'whiskerpad_{os.path.basename(video_name).split(".")[0]}.json', 'r') as f:
    whiskerpad_params = json.load(f)

# Enumerate whiskerpads field and find the "Location" value for the whiskerpad that has "FaceSide" value of face_side
for i, whiskerpad in enumerate(whiskerpad_params['whiskerpads']):
    if whiskerpad['FaceSide'].lower() == face_side:
        whiskerpad_location = whiskerpad['Location']
        image_coord = whiskerpad['ImageCoordinates']
        break

# Convert whiskerpad_location to a NumPy array
whiskerpad_location = np.array(whiskerpad_location)

# Get the table index for the first frame, with fid = 0 
first_frame_idx = np.where(summary[:]['fid'] == 0)[0]

# Get the index for the three longest whiskers in the first frame
longest_whiskers = np.argsort(summary[first_frame_idx]['length'])[-3:]

# # Sort them by the face_x coordinate
longest_whiskers = longest_whiskers[np.argsort(summary[first_frame_idx]['follicle_y'][longest_whiskers])]

In [ ]:
longest_whiskers

In [ ]:
# Read in a frame of the video
frame_num = 0
cap = cv2.VideoCapture('whiskers.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()

In [ ]:
first_frame_idx

In [ ]:
# # Print first values of pixels_x and pixels_y for the first frame, for  all whiskers
# print("Pixel_x and Pixels_y values for first frame, for all whiskers:")
# for whisker_id in first_frame_idx:
#     print(pixels_x[whisker_id][0], pixels_y[whisker_id][0])

# # Print the follicle_x and follicle_y values for the first frame, for all whiskers
# print("Follicle_x and Follicle_y values for first frame, for all whiskers:")
# for whisker_id in first_frame_idx:
#     print(summary[whisker_id]['follicle_x'], summary[whisker_id]['follicle_y'])

In [ ]:
# Using index of three longest whiskers, plot the whiskers on the frame
for i, whisker_id in enumerate(longest_whiskers):
    color_index = i % len(colors)
    color = colors[color_index]
    print(f'Whisker ID: {whisker_id}, color: {color}')

    print(pixels_x[whisker_id][0], pixels_y[whisker_id][0])
    for j in range(pixels_x[whisker_id].shape[0]):
        # add the image_coord first two values
        x = int(pixels_x[whisker_id][j]) + image_coord[0]
        y = int(pixels_y[whisker_id][j]) + image_coord[1]
        cv2.circle(frame, (x,y), 2, color, -1)

In [ ]:
# Plot the frame below the cell in the notebook
plt.imshow(frame)

In [ ]:
# Now the same for the right side:
face_side = 'right'
h5file = tables.open_file(f'{os.path.basename(video_name).split(".")[0]}_{face_side}.hdf5', mode='r')
pixels_x = h5file.get_node('/pixels_x')
pixels_y = h5file.get_node('/pixels_y')

# Get the summary data
summary = h5file.get_node('/summary')

# Get unique whisker IDs
whisker_ids = np.unique(summary[:]['wid'])

# Create a different set of colors for up to 20 whiskers
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255),
            (0,255,255), (128,0,0), (0,128,0), (0,0,128), (128,128,0),
            (128,0,128), (0,128,128), (64,0,0), (0,64,0), (0,0,64),
            (64,64,0), (64,0,64), (0,64,64), (192,0,0), (0,192,0)]


In [ ]:
# Enumerate whiskerpads field and find the "Location" value for the whiskerpad that has "FaceSide" value of face_side
for i, whiskerpad in enumerate(whiskerpad_params['whiskerpads']):
    if whiskerpad['FaceSide'].lower() == face_side:
        whiskerpad_location = whiskerpad['Location']
        image_coord = whiskerpad['ImageCoordinates']
        break

# Convert whiskerpad_location to a NumPy array
whiskerpad_location = np.array(whiskerpad_location)

# Get the table index for the first frame, with fid = 0 
first_frame_idx = np.where(summary[:]['fid'] == 0)[0]

# Get the index for the three longest whiskers in the first frame
longest_whiskers = np.argsort(summary[first_frame_idx]['length'])[-3:]

# Sort them by the face_x coordinate
longest_whiskers = longest_whiskers[np.argsort(summary[first_frame_idx]['follicle_y'][longest_whiskers])]

In [ ]:
longest_whiskers

In [ ]:
# Using index of three longest whiskers, plot the whiskers on the frame
for i, whisker_id in enumerate(longest_whiskers):
    color_index = i % len(colors)
    color = colors[color_index]
    for i in range(pixels_x[whisker_id].shape[0]):
        # add the image_coord first two values
        x = int(pixels_x[whisker_id][i]) + image_coord[0]
        y = int(pixels_y[whisker_id][i]) + image_coord[1]
        cv2.circle(frame, (x,y), 2, color, -1)

In [ ]:
# Plot the frame below the cell in the notebook
plt.imshow(frame)

In [ ]:
# Release the resources
cap.release()
h5file.close()